# 搭建一个简单的RAG

目标：走一遍全流程，对RAG具体怎么实现有一个具体的认识；

<img src="RAG.png" alt="Simple RAGo" style="width:550px;height:300px;">
<img src="dataPrepare.png" alt="Data prepare" style="width:550px;height:300px;">


# 向量存储与检索

In [30]:
# 从本地load文件数据
import json
from pathlib import Path
from langchain.docstore.document import Document

class DataLoader:
    def __init__(self, json_file_path):
        self.json_file_path = json_file_path

    def load_data(self):
        data = json.loads(Path(self.json_file_path).read_text())
        return data

    def create_documents(self, length=None):
        data = self.load_data()
        if length is None:
            length = len(data)
        
        documents = [
            Document(
                page_content=self.get_page_content(item),
                metadata=item
            )
            for item in data[:length]
        ]
        return documents

    def get_page_content(self, item):
        return f"{item['title']} {item['author']} {item['publication_date']} {item['description']} {' '.join(item['genres'])}"

In [31]:
# 数据向量化
from langchain.embeddings import HuggingFaceEmbeddings

class EmbeddingManager:
    def __init__(self, model_name):
        self.embeddings = HuggingFaceEmbeddings(model_name=model_name)
    
    def get_embeddings(self):
        return self.embeddings

In [32]:
# 简单的”向量数据库“
from langchain.vectorstores import FAISS
from tqdm import tqdm

class VectorSpaceManager:
    def __init__(self, embedding_manager):
        self.embedding_manager = embedding_manager
        self.embeddings = self.embedding_manager.get_embeddings()

    def create_vector_space(self, documents):
        vector_store = FAISS.from_documents(documents[:2], self.embeddings)

        with tqdm(total=len(documents), desc="Creating vector space") as pbar:
            batch_size = 100
            for i in range(2, len(documents), batch_size):
                batch_documents = documents[i:i+batch_size]
                tempt_vector_store = FAISS.from_documents(batch_documents, self.embeddings)
                vector_store.merge_from(tempt_vector_store)
                pbar.update(len(batch_documents))

        return vector_store

    def save_vector_space(self, vector_store, save_path):
        print(f"Saving vector space to {save_path}...")
        vector_store.save_local(save_path)
        print(f"Finished!")

    def load_vector_space(self, save_path):
        print(f"Lodaing vector space from {save_path}")
        return FAISS.load_local(save_path, self.embeddings, allow_dangerous_deserialization=True)


In [33]:
# 向量存储
def process_data(json_file_path, model_name, save_path, data_loader_class, length=None):
    # 数据向量化模型
    embedding_manager = EmbeddingManager(model_name)

    # ”向量数据库“
    vector_space_manager = VectorSpaceManager(embedding_manager)

    # 从本地加载数据
    data_loader = data_loader_class(json_file_path)
    documents = data_loader.create_documents(length=length)

    # 调用”向量数据库“将本地数据向量化
    vector_store = vector_space_manager.create_vector_space(documents)
    # 存储向量化后的数据
    vector_space_manager.save_vector_space(vector_store, save_path)

    # 从”向量数据库“中查询数据
    vector_store = vector_space_manager.load_vector_space(save_path)
    query = "找本划水的书"
    search_results = vector_store.search(query, k=2, search_type="similarity")
    print("here is the search results:", search_results)

# 测试用例
if __name__ == "__main__":
    # Book
    json_file_path = 'data/BookSummaries/book.json'  # Replace with the actual JSON file path
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    save_path = 'data/book_vector_store'  # Replace with the actual save path
    process_data(json_file_path, model_name, save_path, DataLoader, 100)

Creating vector space:  98%|█████████▊| 98/100 [00:00<00:00, 103.42it/s]

Saving vector space to data/book_vector_store...
Finished!
Lodaing vector space from data/book_vector_store
here is the search results: [Document(page_content="A book not exist Liu Jia 2024 这是一本没有什么内容的书 Children's literature Juvenile fantasy Adventure novel Speculative fiction Fantasy Fiction", metadata={'id': '30292', 'title': 'A book not exist', 'author': 'Liu Jia', 'publication_date': '2024', 'genres': ["Children's literature", 'Juvenile fantasy', 'Adventure novel', 'Speculative fiction', 'Fantasy', 'Fiction'], 'description': '这是一本没有什么内容的书'}), Document(page_content='The Pit and the Pendulum Nancy Kilpatrick 1842  The story takes place during the Spanish Inquisition. At the beginning of the story an unnamed narrator is brought to trial before various sinister judges. Poe provides no explanation of why he is there or for what he has been arrested. Before him are seven tall white candles on a table, and, as they melt, his hopes of survival also diminish. He is condemned to death and fi

# 根据用户输入，检索信息提供给大模型

In [34]:
# 检索信息并拼接prompt

from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS  
from langchain.agents import Tool
from langchain import PromptTemplate

class ToolManager:
    def __init__(self, llm, books_vector_path, embeddings):
        self.llm = llm
        self.books_vector_path = books_vector_path
        self.embeddings = embeddings
        self.tools = self._initialize_tool()

    def _initialize_tool(self):
        # 链接”向量数据库“
        books_vector_store = FAISS.load_local(self.books_vector_path, self.embeddings, allow_dangerous_deserialization=True)

        # 定义提示词模版
        prompt_template = """If the context is not relevant, 
        please answer the question by using your own knowledge about the topic
        
        {context}
        
        Question: {question}
        """
        PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

        # 使用RetrievalQA检索向量
        books_qa = RetrievalQA.from_chain_type(llm=self.llm, chain_type="stuff", retriever=books_vector_store.as_retriever(search_kwargs={"k": 3}), chain_type_kwargs={"prompt": PROMPT})

        return {
            "books": Tool(name="BooksTool", func=books_qa.run, description="Retrieve book information.")
        }
    def get_tool(self):
       return self.tools.get("books")

In [35]:
# 提供用户交互界面 & 调用大模型
from langchain.memory import ConversationBufferMemory
from langchain.agents import ConversationalChatAgent, AgentExecutor
import time

class ChatAgent:
    def __init__(self, llm, tool_manager):
        self.llm = llm
        self.tool_manager = tool_manager
        self.memory = ConversationBufferMemory(memory_key="chat_history",input_key="input", return_messages=True)
        self.agent = ConversationalChatAgent.from_llm_and_tools(llm=self.llm, tools=list(self.tool_manager.tools.values()), system_message="You are a smart assistant whose main goal is to recommend amazing books and movies to users. Provide helpful, **short** and concise recommendations with a touch of fun!")
        self.chat_agent = AgentExecutor.from_agent_and_tools(agent=self.agent, tools=list(self.tool_manager.tools.values()), verbose=True, memory=self.memory)

    def get_response(self, query):

        try:
            response = self.chat_agent.run(input=query)
        except ValueError as e:
            response = str(e)

        return {"answer": response}


In [36]:
#  配置大模型

import os
from langchain.chat_models import ChatOpenAI

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', '你的key')
llm = ChatOpenAI(openai_api_base="https://api.lingyiwanwu.com/v1", openai_api_key=OPENAI_API_KEY, model="yi-34b-chat-0205",temperature=0.7)

In [37]:
# 测试

# from vector_space import EmbeddingManager

# Initialize components
book_vector_store_path = "data/book_vector_store"
embeddings_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = EmbeddingManager(embeddings_model_name).get_embeddings()
tool_manager = ToolManager(llm, book_vector_store_path,embeddings)
chat_agent = ChatAgent(llm, tool_manager)

print("Chatbot is ready to talk! Type 'quit' to exit.")
    
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break

    response = chat_agent.get_response(user_input)
    print(f"You: {user_input}")
    print(f"Chatbot: {response['answer']}")

Chatbot is ready to talk! Type 'quit' to exit.


> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "The Art of War by Sun Tzu"
}

> Finished chain.
You: 找一本书里全是虚头巴脑的假大空的书
Chatbot: The Art of War by Sun Tzu


> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "《刘佳作品集》"
}

> Finished chain.
You: 找一本刘佳写的，没啥内容的书
Chatbot: 《刘佳作品集》
